# 融合检索

本文介绍的代码实现了一个融合检索系统，它结合了基于向量的相似性搜索和基于关键词的BM25检索。这种方法旨在利用两种方法的优势，提高文档检索的整体质量和相关性。让搜索结果更加精准。

传统的检索方法通常依赖于语义理解（基于向量）或关键词匹配（BM25）。每种方法都有其优点和缺点。融合检索旨在结合这些方法，创建一个更稳健、更准确的检索系统，有效处理更广泛的查询。

## 关键步骤
* 文档处理与文本分块：将文档转换为文本，并将其分割成有重叠的块，每块都带有索引标签。
* 向量存储创建：使用Minimax的嵌入技术为这些文本块创建向量表示，并用Qdrant构建向量存储。
* 创建BM25索引：为关键词搜索创建索引，以实现快速检索。
* 自定义融合检索功能：结合向量和关键词搜索方法，实现更全面的检索。

## 优势
* 提高检索质量：系统能够同时捕捉概念相似性和关键词匹配，提供更准确的搜索结果。
* 灵活性：通过调整alpha参数，可以根据不同的查询需求平衡向量和关键词搜索的权重。
* 稳健性：结合两种方法，可以处理更广泛的查询，减少单一方法的不足。
* 可定制性：系统可以根据需要轻松更换向量存储或关键词检索方法。

首先，我们需要导入一些Python库，并从环境变量文件中加载必要的配置。

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["MINIMAX_GROUP_ID"] = os.getenv("MINIMAX_GROUP_ID")
os.environ["MINIMAX_API_KEY"] = os.getenv("MINIMAX_API_KEY")

from typing import List
from tools import show_context
import numpy as np
from rank_bm25 import BM25Okapi

from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings import MiniMaxEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### 第一步：加载文档并提取文档内容
使用Docx2txtLoader类加载文档，这个类需要docx2txt库支持。

In [3]:
path = "../data/二十届三中全会.docx"
loader = Docx2txtLoader(path)
documents = loader.load()

#### 第二步：文本分块
将文本切割成小块

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
docs = text_splitter.split_documents(documents)
chunk_count = len(docs)

#### 第三步：创建向量数据库
使用MiniMaxEmbeddings和QdrantVectorStore创建向量表示，并存储在本地的Qdrant数据库中。

In [5]:
embeddings = MiniMaxEmbeddings()
vectorstore = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    prefer_grpc=True,
    collection_name="003"
)


#### 第四步：创建BM25索引
使用文本块创建BM25索引，以实现基于关键词的快速检索。

In [16]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

for doc in docs:
    doc.page_content = doc.page_content.replace('\t', ' ') 

bm25 = create_bm25_index(docs) 

#### 第五步：定义融合检索函数
该函数结合了向量搜索和BM25搜索的结果，通过归一化和加权组合得分，返回最相关的文档。

In [18]:
def fusion_retrieval(vectorstore, bm25, query: str, chunk_count: int, k: int = 5, alpha: float = 0.5) -> List[Document]:
    # Step 1: 从vectorstore中获取所有文档
    all_docs = vectorstore.similarity_search("", k=chunk_count)

    # Step 2: 获取查询的BM25分数
    bm25_scores = bm25.get_scores(query.split())

    # Step 3: 使用向量搜索获取相关文档及其分数
    vector_results = vectorstore.similarity_search_with_score(query, k=chunk_count)

    # Step 4: 归一化分数
    vector_scores = np.array([score for _, score in vector_results])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores))

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))

    # Step 5: 结合分数
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores

    # Step 6: 排序文档
    sorted_indices = np.argsort(combined_scores)[::-1]

    # Step 7: 返回前k个文档
    return [all_docs[i] for i in sorted_indices[:k]]

最后，我们使用定义好的融合检索函数，根据用户查询返回最相关的文档。

In [29]:
query = "深化财税体制改革"

top_docs = fusion_retrieval(vectorstore, bm25, query, chunk_count, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

Context 1:
（35）健全公正执法司法体制机制。健全监察机关、公安机关、检察机关、审判机关、司法行政机关各司其职，监察权、侦查权、检察权、审判权、执行权相互配合、相互制约的体制机制，确保执法司法各环节全过程在有效制约监督下运行。深化审判权和执行权分离改革，健全国家执行体制，强化当事人、检察机关和社会公众对执行活动的全程监督。完善执法司法救济保护制度，完善国家赔偿制度。深化和规范司法公开，落实和完善司法责任制。规范专门法院设置。深化行政案件级别管辖、集中管辖、异地管辖改革。构建协同高效的警务体制机制，推进地方公安机关机构编制管理改革，继续推进民航公安机关和海关缉私部门管理体制改革。规范警务辅助人员管理制度。

坚持正确人权观，加强人权执法司法保障，完善事前审查、事中监督、事后纠正等工作机制，完善涉及公民人身权利强制措施以及查封、扣押、冻结等强制措施的制度，依法查处利用职权徇私枉法、非法拘禁、刑讯逼供等犯罪行为。推进刑事案件律师辩护全覆盖。建立轻微犯罪记录封存制度。


Context 2:
（56）深化跨军地改革。健全一体化国家战略体系和能力建设工作机制，完善涉军决策议事协调体制机制。健全国防建设军事需求提报和军地对接机制，完善国防动员体系。深化国防科技工业体制改革，优化国防科技工业布局，改进武器装备采购制度，建立军品设计回报机制，构建武器装备现代化管理体系。完善军地标准化工作统筹机制。加强航天、军贸等领域建设和管理统筹。优化边海防领导管理体制机制，完善党政军警民合力治边机制。深化民兵制度改革。完善双拥工作机制。

十五、提高党对进一步全面深化改革、推进中国式现代化的领导水平

党的领导是进一步全面深化改革、推进中国式现代化的根本保证。必须深刻领悟“两个确立”的决定性意义，增强“四个意识”、坚定“四个自信”、做到“两个维护”，保持以党的自我革命引领社会革命的高度自觉，坚持用改革精神和严的标准管党治党，完善党的自我革命制度规范体系，不断推进党的自我净化、自我完善、自我革新、自我提高，确保党始终成为中国特色社会主义事业的坚强领导核心。


Context 3:
（2）进一步全面深化改革的指导思想。坚持马克思列宁主义、毛泽东思想、邓小平理论、“三个代表”重要思想、科学发展观，全面贯彻习近平新时代中国特色社会主义思想，深入学习贯彻习近平总书记关于全面深化改革的一系列新

C:\Users\39077\AppData\Local\Temp\ipykernel_4860\106719400.py:15: RuntimeWarning: invalid value encountered in divide
  bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))
